In [1]:
#pip install python-dotenv

In [56]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import csv
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import errorcode

In [57]:
load_dotenv()

True

In [58]:
sp_clave = os.getenv("miriam_id_spotify")
sp_contraseña = os.getenv("miriam_secret_spotify")

In [59]:
#Autentificación:
auth_manager = sp = SpotifyClientCredentials(client_id=sp_clave,
                                             client_secret=sp_contraseña)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [60]:
datos_pop = sp.search(q="genre:pop, year:2019", type="track", limit=50, offset=0)

In [61]:
canciones_pop = {"nombre_artista": [], "nombre_cancion": [], "año_lanzamiento": [], 'género_musical': [], 'tipo': []}
for anio in range(2019,2024):
    for offset in range(0, 250, 50):
        datos_pop = sp.search(q=f"genre:pop, year:{anio}", type="track", limit=50, offset=offset)
        for cancion in datos_pop["tracks"]["items"]:
            canciones_pop["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
            canciones_pop["nombre_cancion"].append(cancion["name"])
            canciones_pop["año_lanzamiento"].append(cancion["album"]["release_date"][0:4])
            canciones_pop["género_musical"].append('pop')
            canciones_pop["tipo"].append(cancion["type"])
df_pop = pd.DataFrame(canciones_pop)
df_pop

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,Various Artists,Un boomerang,2019,pop,track
1,Los Rebeldes,Land of Thousand Dances - Remasterizado,2019,pop,track
2,Various Artists,No te enamores,2019,pop,track
3,Various Artists,Walk Me Home,2019,pop,track
4,VINAI,Break the Beat - Radio Edit,2019,pop,track
...,...,...,...,...,...
1245,Joy Sarkar,Poila Shobaar Thaak,2023,pop,track
1246,antonio pereira,Codajás,2023,pop,track
1247,Meena Rana,Bhair Twe Bhair,2023,pop,track
1248,POTRET,Belum Tentu - Remastered 2023,2023,pop,track


In [62]:
df_pop.to_csv('pop_spotifydatabase.csv')

In [63]:
df_values=pd.read_csv("pop_spotifydatabase.csv",index_col=0)

In [64]:
values = list(df_values.itertuples(index=False, name=None))

In [65]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='prueba_musictrends')


mycursor = cnx.cursor()
mycursor.execute("SELECT id_artista, nombre_artista FROM lastfm")
artistas = mycursor.fetchall()
# Diccionario: nombre_artista.lower() -> id_artista para relacionar el id en las dos tablas
diccionario_artistas = {nombre.lower(): id for id, nombre in artistas}

In [ ]:
# insertando todos los artistas:
valores_spotify = []

for fila in values:
    nombre_artista = fila[0].strip().lower()
    id_artista = diccionario_artistas.get(nombre_artista)

    if id_artista is not None:
        # Se encontró ID del artista
        valores_spotify.append((id_artista, *fila))
    else:
        # No se encontró el artista: se inserta con id_artista = None (SQL NULL)
        valores_spotify.append((None, *fila)) # captura el resto de los valores de la lista
        #print(f"⚠️ Artista no encontrado en lastfm (se insertará con id_artista = NULL): {nombre_artista}")

In [67]:
len(valores_spotify)

1250

In [68]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='prueba_musictrends')


mycursor = cnx.cursor()
sql = "INSERT INTO Spotify (id_artista, nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s, %s,%s)"
val = values

try: 
    mycursor.executemany(sql, valores_spotify)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1250 registro/s insertado/s.
